This was for if wanting to produce mutual protection figure with variance constant.

Not using any more I think!!

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from scipy.optimize import minimize

from polymodel.config import Config

from polymodel.run import simulations_run, no_joblib_multiple_run

from polymodel.utils import (
    get_dist_mean
)

from polymodel.consts import (
    DEFAULT_I0,
    DEFAULT_BETA,
    MUTATION_PROP,
    FUNG_MUTATION_SCALE,
    HOST_MUTATION_SCALE,
    DEFAULT_P
)

from polymodel.params import PARAMS

# from polymodel.utils import (
#     host_growth_function,
#     edge_values
# )

# Draft of cluster stuff

In [ ]:
def find_host_var(mu, b):
    a = (b*mu)/(1-mu)
    
    var = a*b / ((a+b+1) * (a+b)**2)
    return var

In [ ]:
def optimise_b_given_mu_var(host_mean, host_var):
    res = minimize(
        lambda x: (find_host_var(host_mean, x) - host_var)**2,
        [5],
        tol=1e-10
    )
    
    return res

In [ ]:
def get_df_of_means_and_variances(means):
    
    cf = Config(
        'single',
        n_k=100,
        n_l=100,
        sprays=[1, 2, 3],
        host_on=[True],
        n_years=15,
        mutation_proportion=MUTATION_PROP,
        mutation_scale_fung=FUNG_MUTATION_SCALE*DEFAULT_P,
        mutation_scale_host=HOST_MUTATION_SCALE*DEFAULT_P,
    )
    
    b_vals = np.zeros(len(means))
    opt_vals = np.zeros(len(means))
    success = []
    
    for ii, mean in enumerate(means):
    
        host_var = find_host_var(cf.l_mu, cf.l_b)
        res = optimise_b_given_mu_var(mean, host_var)
        
        if res.success:
            b_vals[ii] = res.x[0]
            opt_vals[ii] = res.fun
            success.append(True)
            
        else:
            success.append(False)
        
        
    
    df_out = pd.DataFrame(dict(
        b=b_vals,
        mu=means,
        opt=opt_vals,
        success=success,
    ))
                               
    return df_out

In [ ]:
df = get_df_of_means_and_variances(np.linspace(0.01,0.99, 99))

In [ ]:
df.loc[~df.success]

In [ ]:
df.plot.scatter(x='mu', y='b')